<h1><center>MPST: A Corpus of Movie Plot Synopses with Tags</center></h1>

In [1]:
import pandas as pd
import sqlite3
import os
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns


from wordcloud import WordCloud
import numpy as np
import re

from bs4 import BeautifulSoup
import tqdm

import nltk
from itertools import combinations
#from toolz import compose

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')

from nltk.corpus import words
from nltk.tokenize import sent_tokenize
stemmer = SnowballStemmer('english')
from scipy.sparse import coo_matrix, hstack
from tqdm import tqdm

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import f1_score,precision_score,recall_score,hamming_loss

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MultiLabelBinarizer

import xgboost as xgb

[nltk_data] Downloading package punkt to
[nltk_data]     /home/krushithreddy0817/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
data = pd.read_csv("cleaned_data.csv")
data.head()

,title,plot_synopsis,tags,split,synopsis_source,cnt_dup,tag_count,synopsis_count,synopsis_sent_count,CleanedSynopsis
0,$,"Set in Hamburg, West Germany, several criminal...",murder,test,imdb,1,1,648,26,set hamburg west germani sever crimin take adv...
1,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,1,1,353,14,grader name griffin bing decid gather entir gr...
2,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,wikipedia,1,4,699,39,gari hook new recruit british armi take leav m...
3,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,wikipedia,1,2,665,41,sergeant dragon jacki chan part hong kong mari...
4,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,wikipedia,1,6,1694,140,pretoria south africa major charl bolton rod m...


In [4]:
conn = sqlite3.connect('cleaned_data.db')
data.to_sql('cleaned_data', conn, if_exists='replace', index=False)
train = pd.read_sql("Select * From cleaned_data where split = 'train' OR split='val'",conn)
test =  pd.read_sql("Select * From cleaned_data where split = 'test'",conn)
conn.close()

In [5]:
print(train.head())
print(train.shape)

              title                                      plot_synopsis  \
0           $windle  A 6th grader named Griffin Bing decides to gat...   
1               '71  Gary Hook, a new recruit to the British Army, ...   
2       'A' gai wak  Sergeant Dragon Ma (Jackie Chan) is part of th...   
3  'Breaker' Morant  In Pretoria, South Africa, in 1902, Major Char...   
4           'C'-Man  Customs Investigator Cliff Holden (Dean Jagger...   

                                                tags  split synopsis_source  \
0                                          flashback  train       wikipedia   
1            suspenseful, neo noir, murder, violence  train       wikipedia   
2                                     cult, violence  train       wikipedia   
3  murder, anti war, violence, flashback, tragedy...  train       wikipedia   
4                                             murder  train            imdb   

   cnt_dup  tag_count  synopsis_count  synopsis_sent_count  \
0        1        

In [11]:
print(test.head())
print(test.shape)

                                             title  \
0                                                $   
1                            'Crocodile' Dundee II   
2                      'Hukkunud Alpinisti' hotell   
3  'Northwest Passage' (Book I -- Rogers' Rangers)   
4                             (500) Days of Summer   

                                       plot_synopsis  \
0  Set in Hamburg, West Germany, several criminal...   
1  A year has passed since the events of Crocodil...   
2  Inspector Glebsky arrives at the hotel "Dead M...   
3  === Book 1 ===\nLangdon Towne is a young Congr...   
4  (500) Days of Summer is presented in a non-chr...   

                                                tags split synopsis_source  \
0                                             murder  test            imdb   
1                                             murder  test       wikipedia   
2                             cult, neo noir, murder  test       wikipedia   
3           tragedy, reven

# Tag vectorizaton:

In [6]:
tag_vect = CountVectorizer(max_features=5,tokenizer = lambda x: str(x).split(", "))
y_train = tag_vect.fit_transform(train['tags'])
y_test = tag_vect.transform(test['tags'])
print(tag_vect.get_feature_names())

['cult', 'flashback', 'murder', 'romantic', 'violence']


In [7]:
names = tag_vect.get_feature_names()

# CountVectorizing with words and uni grams:

In [14]:
vectorize=CountVectorizer()
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

In [15]:
print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 78666)   (11027, 5)
(2730, 78666)   (2730, 5)


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  35 out of  45 | elapsed:    5.5s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  40 out of  45 | elapsed:    6.3s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    6.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    6.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.48119998679643167


In [19]:
from sklearn.metrics import classification_report
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4773, Recall: 0.5197, F1-measure: 0.4976
              precision    recall  f1-score   support

        cult       0.36      0.36      0.36       507
   flashback       0.27      0.25      0.26       549
      murder       0.60      0.66      0.63      1043
    romantic       0.42      0.50      0.46       534
    violence       0.55      0.63      0.58       827

   micro avg       0.48      0.52      0.50      3460
   macro avg       0.44      0.48      0.46      3460
weighted avg       0.47      0.52      0.49      3460
 samples avg       0.38      0.40      0.36      3460



In [20]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    6.5s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    7.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    7.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.01, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]


In [21]:
grid_search_cv.best_score_

0.4635168307036294

In [22]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.01,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5128, Recall: 0.3922, F1-measure: 0.4445
              precision    recall  f1-score   support

        cult       0.38      0.15      0.22       507
   flashback       0.29      0.25      0.27       549
      murder       0.64      0.52      0.58      1043
    romantic       0.48      0.35      0.41       534
    violence       0.56      0.49      0.53       827

   micro avg       0.51      0.39      0.44      3460
   macro avg       0.47      0.35      0.40      3460
weighted avg       0.50      0.39      0.44      3460
 samples avg       0.33      0.30      0.29      3460



In [26]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    4.8s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    5.4s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    5.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.456738758582612


In [27]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5974, Recall: 0.3500, F1-measure: 0.4414
              precision    recall  f1-score   support

        cult       0.38      0.12      0.18       507
   flashback       0.37      0.13      0.19       549
      murder       0.67      0.59      0.63      1043
    romantic       0.55      0.27      0.36       534
    violence       0.65      0.39      0.48       827

   micro avg       0.60      0.35      0.44      3460
   macro avg       0.52      0.30      0.37      3460
weighted avg       0.56      0.35      0.42      3460
 samples avg       0.32      0.27      0.27      3460



# CountVectorizing with words and bi grams:

In [29]:
vectorize=CountVectorizer(ngram_range=(2,2),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 89722)   (11027, 5)
(2730, 89722)   (2730, 5)


In [34]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  13 out of  25 | elapsed:    0.8s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:    1.0s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:    1.2s remaining:    0.4s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.5064083226624995


[Parallel(n_jobs=-1)]: Done  22 out of  25 | elapsed:    1.2s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    1.3s finished


In [35]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5160, Recall: 0.4882, F1-measure: 0.5017
              precision    recall  f1-score   support

        cult       0.36      0.23      0.28       507
   flashback       0.35      0.20      0.26       549
      murder       0.64      0.67      0.65      1043
    romantic       0.42      0.51      0.46       534
    violence       0.55      0.59      0.57       827

   micro avg       0.52      0.49      0.50      3460
   macro avg       0.46      0.44      0.45      3460
weighted avg       0.50      0.49      0.49      3460
 samples avg       0.38      0.37      0.36      3460



In [36]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    3.9s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    4.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    4.3s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4244473126376398


In [37]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.0001,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5099, Recall: 0.3578, F1-measure: 0.4205
              precision    recall  f1-score   support

        cult       0.33      0.18      0.23       507
   flashback       0.35      0.19      0.25       549
      murder       0.61      0.51      0.56      1043
    romantic       0.43      0.26      0.33       534
    violence       0.56      0.44      0.50       827

   micro avg       0.51      0.36      0.42      3460
   macro avg       0.46      0.32      0.37      3460
weighted avg       0.49      0.36      0.41      3460
 samples avg       0.29      0.26      0.25      3460



In [38]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    2.6s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    2.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    2.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.42545028451221273


In [39]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.001,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5491, Recall: 0.3428, F1-measure: 0.4221
              precision    recall  f1-score   support

        cult       0.37      0.15      0.21       507
   flashback       0.39      0.17      0.24       549
      murder       0.63      0.51      0.57      1043
    romantic       0.49      0.25      0.33       534
    violence       0.59      0.42      0.49       827

   micro avg       0.55      0.34      0.42      3460
   macro avg       0.49      0.30      0.37      3460
weighted avg       0.52      0.34      0.41      3460
 samples avg       0.28      0.25      0.24      3460



# CountVectorizing with words and tri grams:

In [42]:
vectorize=CountVectorizer(ngram_range=(3,3),min_df=0.00005,max_features=200000)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 200000)   (11027, 5)
(2730, 200000)   (2730, 5)


In [43]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    2.6s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    2.7s remaining:    0.4s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4218685019272745


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    3.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    3.0s finished


In [44]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4648, Recall: 0.3506, F1-measure: 0.3997
              precision    recall  f1-score   support

        cult       0.34      0.16      0.22       507
   flashback       0.26      0.16      0.20       549
      murder       0.54      0.61      0.57      1043
    romantic       0.34      0.17      0.23       534
    violence       0.53      0.38      0.44       827

   micro avg       0.46      0.35      0.40      3460
   macro avg       0.40      0.30      0.33      3460
weighted avg       0.43      0.35      0.38      3460
 samples avg       0.29      0.26      0.25      3460



In [46]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    2.2s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    2.4s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    2.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.30817407908054795


In [47]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4729, Recall: 0.2092, F1-measure: 0.2901
              precision    recall  f1-score   support

        cult       0.39      0.07      0.13       507
   flashback       0.31      0.08      0.13       549
      murder       0.53      0.37      0.44      1043
    romantic       0.36      0.10      0.16       534
    violence       0.48      0.25      0.33       827

   micro avg       0.47      0.21      0.29      3460
   macro avg       0.42      0.17      0.23      3460
weighted avg       0.44      0.21      0.27      3460
 samples avg       0.19      0.15      0.16      3460



In [48]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    1.0s remaining:    0.3s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.31994673261456696


[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    1.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    1.2s finished


In [49]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4721, Recall: 0.2347, F1-measure: 0.3135
              precision    recall  f1-score   support

        cult       0.41      0.09      0.15       507
   flashback       0.30      0.09      0.14       549
      murder       0.54      0.42      0.47      1043
    romantic       0.37      0.17      0.23       534
    violence       0.49      0.23      0.31       827

   micro avg       0.47      0.23      0.31      3460
   macro avg       0.42      0.20      0.26      3460
weighted avg       0.44      0.23      0.30      3460
 samples avg       0.21      0.17      0.18      3460



# CountVectorizing with words and uni,bi grams:

In [50]:
vectorize=CountVectorizer(ngram_range=(1,2),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 109114)   (11027, 5)
(2730, 109114)   (2730, 5)


In [51]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    5.0s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    5.8s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    6.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    6.2s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.5112386180635963


In [52]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5008, Recall: 0.5390, F1-measure: 0.5192
              precision    recall  f1-score   support

        cult       0.36      0.34      0.35       507
   flashback       0.31      0.26      0.28       549
      murder       0.65      0.68      0.66      1043
    romantic       0.43      0.58      0.49       534
    violence       0.56      0.64      0.60       827

   micro avg       0.50      0.54      0.52      3460
   macro avg       0.46      0.50      0.48      3460
weighted avg       0.50      0.54      0.51      3460
 samples avg       0.39      0.41      0.37      3460



In [53]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    6.3s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    7.4s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    7.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    7.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.01, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.459049045715355


In [54]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.01,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5078, Recall: 0.3968, F1-measure: 0.4455
              precision    recall  f1-score   support

        cult       0.34      0.16      0.22       507
   flashback       0.29      0.27      0.28       549
      murder       0.63      0.53      0.58      1043
    romantic       0.47      0.39      0.43       534
    violence       0.60      0.45      0.52       827

   micro avg       0.51      0.40      0.45      3460
   macro avg       0.47      0.36      0.40      3460
weighted avg       0.50      0.40      0.44      3460
 samples avg       0.33      0.30      0.29      3460



In [55]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    4.7s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    5.0s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.6s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4604806856578431


In [56]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6243, Recall: 0.3549, F1-measure: 0.4526
              precision    recall  f1-score   support

        cult       0.46      0.16      0.23       507
   flashback       0.38      0.07      0.13       549
      murder       0.67      0.62      0.64      1043
    romantic       0.58      0.26      0.36       534
    violence       0.68      0.39      0.50       827

   micro avg       0.62      0.35      0.45      3460
   macro avg       0.55      0.30      0.37      3460
weighted avg       0.58      0.35      0.42      3460
 samples avg       0.33      0.27      0.28      3460



# CountVectorizing with words and uni,bi,tri grams:

In [63]:
vectorize=CountVectorizer(ngram_range=(1,3),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 114151)   (11027, 5)
(2730, 114151)   (2730, 5)


In [64]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    5.8s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    6.5s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    7.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    7.0s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.5111489199548436


In [65]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5028, Recall: 0.5361, F1-measure: 0.5190
              precision    recall  f1-score   support

        cult       0.37      0.34      0.35       507
   flashback       0.32      0.27      0.29       549
      murder       0.64      0.67      0.66      1043
    romantic       0.42      0.57      0.49       534
    violence       0.56      0.63      0.60       827

   micro avg       0.50      0.54      0.52      3460
   macro avg       0.46      0.50      0.48      3460
weighted avg       0.50      0.54      0.51      3460
 samples avg       0.39      0.41      0.37      3460



In [66]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    6.6s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    7.9s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    8.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    8.0s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4578162209987625


In [67]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)

predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6259, Recall: 0.3708, F1-measure: 0.4657
              precision    recall  f1-score   support

        cult       0.46      0.11      0.18       507
   flashback       0.40      0.05      0.08       549
      murder       0.66      0.62      0.64      1043
    romantic       0.58      0.28      0.37       534
    violence       0.64      0.50      0.56       827

   micro avg       0.63      0.37      0.47      3460
   macro avg       0.55      0.31      0.37      3460
weighted avg       0.57      0.37      0.42      3460
 samples avg       0.34      0.28      0.29      3460



In [68]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    4.4s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    5.5s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4573853058896979


In [69]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6071, Recall: 0.3751, F1-measure: 0.4637
              precision    recall  f1-score   support

        cult       0.42      0.14      0.21       507
   flashback       0.34      0.04      0.07       549
      murder       0.64      0.66      0.65      1043
    romantic       0.56      0.27      0.36       534
    violence       0.65      0.46      0.54       827

   micro avg       0.61      0.38      0.46      3460
   macro avg       0.52      0.31      0.37      3460
weighted avg       0.55      0.38      0.42      3460
 samples avg       0.35      0.29      0.30      3460



# CountVectorizing with chars and tri grams:

In [71]:
vectorize=CountVectorizer(ngram_range=(3,3),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 10726)   (11027, 5)
(2730, 10726)   (2730, 5)


In [73]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:    9.8s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   10.9s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   11.2s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=2.5, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4907140280954945


In [74]:
clf = OneVsRestClassifier(MultinomialNB(alpha=2.5),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4397, Recall: 0.5815, F1-measure: 0.5007
              precision    recall  f1-score   support

        cult       0.31      0.48      0.37       507
   flashback       0.26      0.35      0.30       549
      murder       0.61      0.65      0.63      1043
    romantic       0.39      0.65      0.48       534
    violence       0.53      0.67      0.59       827

   micro avg       0.44      0.58      0.50      3460
   macro avg       0.42      0.56      0.48      3460
weighted avg       0.46      0.58      0.51      3460
 samples avg       0.38      0.45      0.38      3460



In [75]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   13.0s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   15.7s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   15.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   15.8s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False),
          n_jobs=None))]
0.44703851462712957


In [76]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5285, Recall: 0.4208, F1-measure: 0.4685
              precision    recall  f1-score   support

        cult       0.35      0.25      0.29       507
   flashback       0.32      0.10      0.15       549
      murder       0.61      0.71      0.66      1043
    romantic       0.43      0.46      0.45       534
    violence       0.65      0.35      0.46       827

   micro avg       0.53      0.42      0.47      3460
   macro avg       0.47      0.37      0.40      3460
weighted avg       0.51      0.42      0.44      3460
 samples avg       0.36      0.33      0.32      3460



In [77]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    9.4s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   11.7s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   12.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   12.0s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4354080374389804


In [78]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.0001,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4733, Recall: 0.4040, F1-measure: 0.4359
              precision    recall  f1-score   support

        cult       0.32      0.41      0.36       507
   flashback       0.27      0.02      0.03       549
      murder       0.62      0.49      0.55      1043
    romantic       0.48      0.20      0.28       534
    violence       0.46      0.68      0.55       827

   micro avg       0.47      0.40      0.44      3460
   macro avg       0.43      0.36      0.35      3460
weighted avg       0.46      0.40      0.40      3460
 samples avg       0.32      0.30      0.29      3460



# CountVectorizing with chars and four grams:

In [79]:
vectorize=CountVectorizer(ngram_range=(4,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 82528)   (11027, 5)
(2730, 82528)   (2730, 5)


In [80]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   21.4s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   25.8s remaining:    5.7s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   28.2s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   28.6s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=2, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4985966981889413


In [81]:
clf = OneVsRestClassifier(MultinomialNB(alpha=2),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4739, Recall: 0.5442, F1-measure: 0.5067
              precision    recall  f1-score   support

        cult       0.34      0.37      0.35       507
   flashback       0.29      0.30      0.30       549
      murder       0.62      0.66      0.64      1043
    romantic       0.41      0.57      0.48       534
    violence       0.55      0.66      0.60       827

   micro avg       0.47      0.54      0.51      3460
   macro avg       0.44      0.51      0.47      3460
weighted avg       0.47      0.54      0.51      3460
 samples avg       0.38      0.42      0.37      3460



In [82]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   24.7s remaining:    8.2s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   29.9s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   30.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   30.3s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4546875019057356


In [83]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5287, Recall: 0.3780, F1-measure: 0.4408
              precision    recall  f1-score   support

        cult       0.34      0.19      0.24       507
   flashback       0.31      0.12      0.17       549
      murder       0.65      0.55      0.59      1043
    romantic       0.43      0.39      0.41       534
    violence       0.61      0.44      0.51       827

   micro avg       0.53      0.38      0.44      3460
   macro avg       0.47      0.34      0.39      3460
weighted avg       0.50      0.38      0.43      3460
 samples avg       0.34      0.30      0.29      3460



In [84]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   18.7s remaining:    6.2s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   23.0s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   23.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   23.3s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.44909550240263113


In [85]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4742, Recall: 0.4656, F1-measure: 0.4699
              precision    recall  f1-score   support

        cult       0.31      0.12      0.18       507
   flashback       0.28      0.34      0.31       549
      murder       0.57      0.68      0.62      1043
    romantic       0.42      0.40      0.41       534
    violence       0.56      0.53      0.54       827

   micro avg       0.47      0.47      0.47      3460
   macro avg       0.43      0.41      0.41      3460
weighted avg       0.46      0.47      0.45      3460
 samples avg       0.36      0.36      0.33      3460



# CountVectorizing with chars and three,four grams:

In [86]:
vectorize=CountVectorizer(ngram_range=(3,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93254)   (11027, 5)
(2730, 93254)   (2730, 5)


In [87]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   28.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   34.9s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   42.2s remaining:    9.3s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   43.4s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   45.6s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4986242988074304


In [88]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4521, Recall: 0.5659, F1-measure: 0.5026
              precision    recall  f1-score   support

        cult       0.31      0.42      0.36       507
   flashback       0.28      0.35      0.31       549
      murder       0.62      0.65      0.63      1043
    romantic       0.39      0.61      0.48       534
    violence       0.54      0.66      0.59       827

   micro avg       0.45      0.57      0.50      3460
   macro avg       0.43      0.54      0.47      3460
weighted avg       0.46      0.57      0.51      3460
 samples avg       0.38      0.43      0.37      3460



In [89]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   40.2s remaining:   13.4s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   48.6s remaining:    6.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   48.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   48.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False),
          n_jobs=None))]
0.4419158675303263


In [90]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5768, Recall: 0.3829, F1-measure: 0.4603
              precision    recall  f1-score   support

        cult       0.36      0.24      0.29       507
   flashback       0.36      0.18      0.24       549
      murder       0.67      0.61      0.64      1043
    romantic       0.61      0.22      0.32       534
    violence       0.64      0.43      0.52       827

   micro avg       0.58      0.38      0.46      3460
   macro avg       0.53      0.33      0.40      3460
weighted avg       0.56      0.38      0.44      3460
 samples avg       0.32      0.29      0.29      3460



In [91]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   28.7s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   29.4s remaining:    9.8s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   35.8s remaining:    5.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   36.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   36.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4528211054398194


In [92]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4617, Recall: 0.4896, F1-measure: 0.4752
              precision    recall  f1-score   support

        cult       0.31      0.45      0.37       507
   flashback       0.34      0.06      0.10       549
      murder       0.55      0.66      0.60      1043
    romantic       0.48      0.25      0.33       534
    violence       0.46      0.74      0.57       827

   micro avg       0.46      0.49      0.48      3460
   macro avg       0.43      0.43      0.39      3460
weighted avg       0.45      0.49      0.44      3460
 samples avg       0.35      0.37      0.33      3460



# CountVectorizing with chars and bi,tri,four grams:

In [101]:
vectorize=CountVectorizer(ngram_range=(2,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93963)   (11027, 5)
(2730, 93963)   (2730, 5)


In [102]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   32.2s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   39.3s remaining:    8.6s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   46.0s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   46.3s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4984526994035465


In [103]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4516, Recall: 0.5691, F1-measure: 0.5036
              precision    recall  f1-score   support

        cult       0.31      0.44      0.37       507
   flashback       0.28      0.35      0.31       549
      murder       0.62      0.65      0.63      1043
    romantic       0.40      0.61      0.48       534
    violence       0.53      0.66      0.59       827

   micro avg       0.45      0.57      0.50      3460
   macro avg       0.43      0.54      0.48      3460
weighted avg       0.46      0.57      0.51      3460
 samples avg       0.38      0.44      0.38      3460



In [104]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   43.0s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   43.4s remaining:   14.5s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   52.8s remaining:    7.5s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   53.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   53.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.43778056242486424


In [105]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4055, Recall: 0.3725, F1-measure: 0.3883
              precision    recall  f1-score   support

        cult       0.32      0.04      0.08       507
   flashback       0.24      0.72      0.36       549
      murder       0.70      0.35      0.47      1043
    romantic       0.53      0.13      0.21       534
    violence       0.56      0.52      0.54       827

   micro avg       0.41      0.37      0.39      3460
   macro avg       0.47      0.35      0.33      3460
weighted avg       0.51      0.37      0.37      3460
 samples avg       0.28      0.28      0.25      3460



In [106]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   32.2s remaining:   10.7s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   39.8s remaining:    5.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   40.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   40.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.45684002261740947


In [107]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.3950, Recall: 0.4454, F1-measure: 0.4187
              precision    recall  f1-score   support

        cult       0.27      0.01      0.01       507
   flashback       0.30      0.09      0.14       549
      murder       0.74      0.33      0.46      1043
    romantic       0.29      0.72      0.41       534
    violence       0.39      0.92      0.55       827

   micro avg       0.40      0.45      0.42      3460
   macro avg       0.40      0.41      0.32      3460
weighted avg       0.45      0.45      0.36      3460
 samples avg       0.37      0.35      0.33      3460



# CountVectorizing with chars and one,bi,tri,four grams:

In [93]:
vectorize=CountVectorizer(ngram_range=(1,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93990)   (11027, 5)
(2730, 93990)   (2730, 5)


In [94]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   37.8s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   44.9s remaining:    9.9s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   45.7s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   48.8s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4985583137938495


In [96]:
clf = OneVsRestClassifier(MultinomialNB(alpha=1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4508, Recall: 0.5688, F1-measure: 0.5029
              precision    recall  f1-score   support

        cult       0.31      0.44      0.37       507
   flashback       0.28      0.35      0.31       549
      murder       0.62      0.65      0.63      1043
    romantic       0.39      0.61      0.48       534
    violence       0.53      0.66      0.59       827

   micro avg       0.45      0.57      0.50      3460
   macro avg       0.43      0.54      0.48      3460
weighted avg       0.46      0.57      0.51      3460
 samples avg       0.38      0.44      0.37      3460



In [97]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   31.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   44.7s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   45.7s remaining:   15.2s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   54.1s remaining:    7.7s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   55.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   55.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.1, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.43318323519632623


In [98]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=0.1,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.3698, Recall: 0.4098, F1-measure: 0.3888
              precision    recall  f1-score   support

        cult       0.21      0.93      0.34       507
   flashback       0.00      0.00      0.00       549
      murder       0.70      0.45      0.55      1043
    romantic       0.49      0.46      0.47       534
    violence       0.64      0.28      0.39       827

   micro avg       0.37      0.41      0.39      3460
   macro avg       0.41      0.42      0.35      3460
weighted avg       0.47      0.41      0.38      3460
 samples avg       0.31      0.31      0.29      3460



In [99]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   33.0s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   34.0s remaining:   11.3s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   41.2s remaining:    5.9s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   41.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   41.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=2, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False),
          n_jobs=None))]
0.4420668959397315


In [100]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=2,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.3608, Recall: 0.3806, F1-measure: 0.3705
              precision    recall  f1-score   support

        cult       0.38      0.14      0.21       507
   flashback       0.22      0.87      0.35       549
      murder       0.73      0.28      0.40      1043
    romantic       1.00      0.00      0.00       534
    violence       0.54      0.58      0.56       827

   micro avg       0.36      0.38      0.37      3460
   macro avg       0.57      0.37      0.30      3460
weighted avg       0.59      0.38      0.34      3460
 samples avg       0.27      0.27      0.25      3460



In [113]:
from prettytable import PrettyTable
table = PrettyTable()
print("CountVectorization with word analyzer")
table.field_names=['Model','ngrams','f1_score','hyperparameter']
table.add_row(['MultinomialNB','1,1','0.497','1'])
table.add_row(['SGDClassifier(log)','1,1','0.444','0.01'])
table.add_row(['SGDClassifier(hinge)','1,1','0.441','0.1'])
table.add_row(['MultinomialNB','2,2','0.501','1'])
table.add_row(['SGDClassifier(log)','2,2','0.420','0.0001'])
table.add_row(['SGDClassifier(hinge)','2,2','0.422','0.001'])
table.add_row(['MultinomialNB','3,3','0.399','1'])
table.add_row(['SGDClassifier(log)','3,3','0.290','1e-05'])
table.add_row(['SGDClassifier(hinge)','3,3','0.313','1e-05'])
table.add_row(['MultinomialNB','1,2','0.52','1'])
table.add_row(['SGDClassifier(log)','1,2','0.445','0.01'])
table.add_row(['SGDClassifier(hinge)','1,2','0.452','0.1'])
table.add_row(['MultinomialNB','1,3','0.52','1'])
table.add_row(['SGDClassifier(log)','1,3','0.465','0.1'])
table.add_row(['SGDClassifier(hinge)','1,3','0.463','0.1'])
print(table)

CountVectorization with word analyzer
+----------------------+--------+----------+----------------+
|        Model         | ngrams | f1_score | hyperparameter |
+----------------------+--------+----------+----------------+
|    MultinomialNB     |  1,1   |  0.497   |       1        |
|  SGDClassifier(log)  |  1,1   |  0.444   |      0.01      |
| SGDClassifier(hinge) |  1,1   |  0.441   |      0.1       |
|    MultinomialNB     |  2,2   |  0.501   |       1        |
|  SGDClassifier(log)  |  2,2   |  0.420   |     0.0001     |
| SGDClassifier(hinge) |  2,2   |  0.422   |     0.001      |
|    MultinomialNB     |  3,3   |  0.399   |       1        |
|  SGDClassifier(log)  |  3,3   |  0.290   |     1e-05      |
| SGDClassifier(hinge) |  3,3   |  0.313   |     1e-05      |
|    MultinomialNB     |  1,2   |   0.52   |       1        |
|  SGDClassifier(log)  |  1,2   |  0.445   |      0.01      |
| SGDClassifier(hinge) |  1,2   |  0.452   |      0.1       |
|    MultinomialNB     |  1,3   

In [115]:
table = PrettyTable()
print("CountVectorization with char analyzer")
table.field_names=['Model','ngrams','f1_score','hyperparameter']
table.add_row(['MultinomialNB','3,3','0.500','2.5'])
table.add_row(['SGDClassifier(log)','3,3','0.468','1'])
table.add_row(['SGDClassifier(hinge)','3,3','0.435','0.0001'])
table.add_row(['MultinomialNB','4,4','0.506','2'])
table.add_row(['SGDClassifier(log)','4,4','0.440','0.1'])
table.add_row(['SGDClassifier(hinge)','4,4','0.469','0.1'])
table.add_row(['MultinomialNB','3,4','0.502','1'])
table.add_row(['SGDClassifier(log)','3,4','0.460','1'])
table.add_row(['SGDClassifier(hinge)','3,4','0.475','0.1'])
table.add_row(['MultinomialNB','2,4','0.503','1'])
table.add_row(['SGDClassifier(log)','2,4','0.388','1e-05'])
table.add_row(['SGDClassifier(hinge)','2,4','0.418','0.1'])
table.add_row(['MultinomialNB','1,4','0.502','1'])
table.add_row(['SGDClassifier(log)','1,4','0.465','0.1'])
table.add_row(['SGDClassifier(hinge)','1,4','0.370','2'])
print(table)

CountVectorization with char analyzer
+----------------------+--------+----------+----------------+
|        Model         | ngrams | f1_score | hyperparameter |
+----------------------+--------+----------+----------------+
|    MultinomialNB     |  3,3   |  0.500   |      2.5       |
|  SGDClassifier(log)  |  3,3   |  0.468   |       1        |
| SGDClassifier(hinge) |  3,3   |  0.435   |     0.0001     |
|    MultinomialNB     |  4,4   |  0.506   |       2        |
|  SGDClassifier(log)  |  4,4   |  0.440   |      0.1       |
| SGDClassifier(hinge) |  4,4   |  0.469   |      0.1       |
|    MultinomialNB     |  3,4   |  0.502   |       1        |
|  SGDClassifier(log)  |  3,4   |  0.460   |       1        |
| SGDClassifier(hinge) |  3,4   |  0.475   |      0.1       |
|    MultinomialNB     |  2,4   |  0.503   |       1        |
|  SGDClassifier(log)  |  2,4   |  0.388   |     1e-05      |
| SGDClassifier(hinge) |  2,4   |  0.418   |      0.1       |
|    MultinomialNB     |  1,4   

# TfidfVectorizing with words and uni grams:

In [29]:
vectorize=TfidfVectorizer()
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 78666)   (11027, 5)
(2730, 78666)   (2730, 5)


In [30]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  35 out of  45 | elapsed:    2.7s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  40 out of  45 | elapsed:    3.0s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    3.1s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.38160486978328884


In [31]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))


Precision: 0.5359, Recall: 0.2760, F1-measure: 0.3644
              precision    recall  f1-score   support

        cult       0.40      0.10      0.16       507
   flashback       0.23      0.05      0.08       549
      murder       0.58      0.47      0.52      1043
    romantic       0.49      0.18      0.26       534
    violence       0.58      0.36      0.44       827

   micro avg       0.54      0.28      0.36      3460
   macro avg       0.46      0.23      0.29      3460
weighted avg       0.49      0.28      0.34      3460
 samples avg       0.27      0.21      0.22      3460



In [32]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
grid_search_cv.best_score_

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    5.6s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    6.4s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    6.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]


0.4341140828561357

In [33]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4758, Recall: 0.4127, F1-measure: 0.4420
              precision    recall  f1-score   support

        cult       0.33      0.25      0.29       507
   flashback       0.31      0.27      0.29       549
      murder       0.59      0.55      0.57      1043
    romantic       0.41      0.33      0.37       534
    violence       0.53      0.49      0.51       827

   micro avg       0.48      0.41      0.44      3460
   macro avg       0.44      0.38      0.40      3460
weighted avg       0.47      0.41      0.44      3460
 samples avg       0.33      0.31      0.30      3460



In [34]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,2,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    5.2s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    5.8s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    5.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4289163468972529


In [35]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4846, Recall: 0.4234, F1-measure: 0.4520
              precision    recall  f1-score   support

        cult       0.34      0.28      0.31       507
   flashback       0.30      0.23      0.26       549
      murder       0.60      0.57      0.58      1043
    romantic       0.43      0.36      0.40       534
    violence       0.54      0.50      0.52       827

   micro avg       0.48      0.42      0.45      3460
   macro avg       0.44      0.39      0.41      3460
weighted avg       0.48      0.42      0.45      3460
 samples avg       0.34      0.32      0.30      3460



# TfidfVectorizing with words and bi grams

In [36]:
vectorize=TfidfVectorizer(ngram_range=(2,2),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 89722)   (11027, 5)
(2730, 89722)   (2730, 5)


In [37]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  13 out of  25 | elapsed:    0.8s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:    0.9s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:    1.0s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  22 out of  25 | elapsed:    1.1s remaining:    0.1s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.43521252699539387


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    1.2s finished


In [38]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.1),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6381, Recall: 0.3139, F1-measure: 0.4208
              precision    recall  f1-score   support

        cult       0.51      0.05      0.09       507
   flashback       0.35      0.02      0.04       549
      murder       0.67      0.56      0.61      1043
    romantic       0.61      0.20      0.30       534
    violence       0.63      0.43      0.51       827

   micro avg       0.64      0.31      0.42      3460
   macro avg       0.55      0.25      0.31      3460
weighted avg       0.58      0.31      0.37      3460
 samples avg       0.29      0.24      0.25      3460



In [39]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    2.5s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    2.9s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    3.1s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.39549557869128854


In [40]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5837, Recall: 0.2893, F1-measure: 0.3869
              precision    recall  f1-score   support

        cult       0.39      0.08      0.14       507
   flashback       0.38      0.09      0.15       549
      murder       0.65      0.50      0.56      1043
    romantic       0.49      0.15      0.23       534
    violence       0.61      0.37      0.46       827

   micro avg       0.58      0.29      0.39      3460
   macro avg       0.50      0.24      0.31      3460
weighted avg       0.53      0.29      0.36      3460
 samples avg       0.27      0.21      0.22      3460



In [41]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    3.1s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    3.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    3.6s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.41209073217149766


In [42]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5357, Recall: 0.3514, F1-measure: 0.4244
              precision    recall  f1-score   support

        cult       0.33      0.14      0.20       507
   flashback       0.39      0.11      0.17       549
      murder       0.63      0.53      0.57      1043
    romantic       0.45      0.25      0.32       534
    violence       0.55      0.49      0.52       827

   micro avg       0.54      0.35      0.42      3460
   macro avg       0.47      0.30      0.36      3460
weighted avg       0.50      0.35      0.40      3460
 samples avg       0.30      0.26      0.26      3460



# TfidfVectorizing with words and tri grams

In [44]:
vectorize=TfidfVectorizer(ngram_range=(3,3),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 5037)   (11027, 5)
(2730, 5037)   (2730, 5)


In [45]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  13 out of  25 | elapsed:    1.5s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:    1.6s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:    1.6s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  22 out of  25 | elapsed:    1.7s remaining:    0.2s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.2909740125066573


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    1.7s finished


In [46]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5087, Recall: 0.1861, F1-measure: 0.2725
              precision    recall  f1-score   support

        cult       0.40      0.04      0.08       507
   flashback       0.25      0.03      0.05       549
      murder       0.54      0.36      0.43      1043
    romantic       0.46      0.11      0.18       534
    violence       0.53      0.20      0.29       827

   micro avg       0.51      0.19      0.27      3460
   macro avg       0.44      0.15      0.21      3460
weighted avg       0.46      0.19      0.25      3460
 samples avg       0.18      0.14      0.15      3460



In [48]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1089s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  28 out of  55 | elapsed:    0.5s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  34 out of  55 | elapsed:    0.6s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  40 out of  55 | elapsed:    0.6s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    0.6s remaining:    0.1s


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.33375960286796164


[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    0.8s finished


In [49]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.3553, Recall: 0.3341, F1-measure: 0.3444
              precision    recall  f1-score   support

        cult       0.23      0.21      0.22       507
   flashback       0.26      0.24      0.25       549
      murder       0.47      0.43      0.45      1043
    romantic       0.28      0.30      0.29       534
    violence       0.40      0.37      0.38       827

   micro avg       0.36      0.33      0.34      3460
   macro avg       0.33      0.31      0.32      3460
weighted avg       0.36      0.33      0.34      3460
 samples avg       0.25      0.25      0.22      3460



In [50]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0725s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  16 out of  55 | elapsed:    0.2s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  28 out of  55 | elapsed:    0.4s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  40 out of  55 | elapsed:    0.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    0.6s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.3358370647517559


In [51]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.3523, Recall: 0.3173, F1-measure: 0.3339
              precision    recall  f1-score   support

        cult       0.23      0.20      0.21       507
   flashback       0.22      0.21      0.22       549
      murder       0.47      0.44      0.46      1043
    romantic       0.30      0.27      0.29       534
    violence       0.40      0.33      0.36       827

   micro avg       0.35      0.32      0.33      3460
   macro avg       0.32      0.29      0.31      3460
weighted avg       0.35      0.32      0.33      3460
 samples avg       0.24      0.23      0.21      3460



# TfidfVectorizing with words and uni,bi grams

In [52]:
vectorize=TfidfVectorizer(ngram_range=(1,2),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 109114)   (11027, 5)
(2730, 109114)   (2730, 5)


In [53]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  13 out of  25 | elapsed:    2.3s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:    2.7s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:    3.1s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  22 out of  25 | elapsed:    3.4s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    4.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    4.0s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.40264630207628344


In [54]:
from sklearn.metrics import classification_report
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6305, Recall: 0.3092, F1-measure: 0.4150
              precision    recall  f1-score   support

        cult       0.52      0.07      0.12       507
   flashback       0.33      0.03      0.05       549
      murder       0.66      0.54      0.59      1043
    romantic       0.60      0.22      0.32       534
    violence       0.64      0.41      0.50       827

   micro avg       0.63      0.31      0.41      3460
   macro avg       0.55      0.25      0.32      3460
weighted avg       0.57      0.31      0.37      3460
 samples avg       0.30      0.24      0.25      3460



In [55]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    8.3s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    9.2s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    9.4s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_i...e, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__estimator__alpha': [3, 3.5, 2, 2.5, 1, 0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [56]:
print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.44404908590584974


In [57]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4866, Recall: 0.4130, F1-measure: 0.4468
              precision    recall  f1-score   support

        cult       0.33      0.27      0.30       507
   flashback       0.31      0.20      0.25       549
      murder       0.60      0.56      0.58      1043
    romantic       0.43      0.35      0.38       534
    violence       0.55      0.50      0.52       827

   micro avg       0.49      0.41      0.45      3460
   macro avg       0.44      0.38      0.41      3460
weighted avg       0.48      0.41      0.44      3460
 samples avg       0.33      0.32      0.30      3460



In [58]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    7.3s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    8.2s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    8.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4461305769525208


In [59]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4951, Recall: 0.4199, F1-measure: 0.4544
              precision    recall  f1-score   support

        cult       0.31      0.26      0.29       507
   flashback       0.32      0.23      0.27       549
      murder       0.60      0.59      0.59      1043
    romantic       0.45      0.35      0.39       534
    violence       0.59      0.47      0.52       827

   micro avg       0.50      0.42      0.45      3460
   macro avg       0.45      0.38      0.41      3460
weighted avg       0.49      0.42      0.45      3460
 samples avg       0.34      0.32      0.31      3460



# TfidfVectorizing with words and uni,bi,tri grams 

In [63]:
vectorize=TfidfVectorizer(ngram_range=(1,3),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 114151)   (11027, 5)
(2730, 114151)   (2730, 5)


In [64]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [2,1.5,1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  24 out of  35 | elapsed:    3.6s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed:    4.3s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  32 out of  35 | elapsed:    4.8s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    4.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.4046635948108246


In [65]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6317, Recall: 0.3113, F1-measure: 0.4170
              precision    recall  f1-score   support

        cult       0.52      0.07      0.12       507
   flashback       0.33      0.03      0.05       549
      murder       0.66      0.55      0.60      1043
    romantic       0.59      0.22      0.32       534
    violence       0.64      0.41      0.50       827

   micro avg       0.63      0.31      0.42      3460
   macro avg       0.55      0.25      0.32      3460
weighted avg       0.57      0.31      0.37      3460
 samples avg       0.30      0.24      0.25      3460



In [66]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    9.0s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:   10.1s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:   10.2s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4429005030403062


In [68]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5128, Recall: 0.3939, F1-measure: 0.4456
              precision    recall  f1-score   support

        cult       0.35      0.20      0.26       507
   flashback       0.33      0.19      0.24       549
      murder       0.60      0.56      0.58      1043
    romantic       0.45      0.32      0.37       534
    violence       0.57      0.48      0.52       827

   micro avg       0.51      0.39      0.45      3460
   macro avg       0.46      0.35      0.40      3460
weighted avg       0.49      0.39      0.43      3460
 samples avg       0.33      0.30      0.29      3460



In [69]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    7.6s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    8.6s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    8.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4423138333125596


In [70]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4907, Recall: 0.4179, F1-measure: 0.4514
              precision    recall  f1-score   support

        cult       0.33      0.23      0.27       507
   flashback       0.32      0.27      0.29       549
      murder       0.62      0.56      0.59      1043
    romantic       0.42      0.38      0.39       534
    violence       0.56      0.48      0.52       827

   micro avg       0.49      0.42      0.45      3460
   macro avg       0.45      0.38      0.41      3460
weighted avg       0.48      0.42      0.45      3460
 samples avg       0.34      0.32      0.30      3460



# TfidfVectorizing with words and uni,bi,tri,four grams

In [71]:
vectorize=TfidfVectorizer(ngram_range=(1,4),min_df=0.0005)
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 114710)   (11027, 5)
(2730, 114710)   (2730, 5)


In [72]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [2,1.5,1,10**-1, 10**-2, 10**-3, 10**-4],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)
print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  24 out of  35 | elapsed:    4.1s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed:    4.7s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  32 out of  35 | elapsed:    5.0s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    5.2s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.40416379234010996


In [73]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6326, Recall: 0.3116, F1-measure: 0.4175
              precision    recall  f1-score   support

        cult       0.52      0.07      0.12       507
   flashback       0.33      0.03      0.05       549
      murder       0.66      0.56      0.60      1043
    romantic       0.59      0.22      0.32       534
    violence       0.64      0.41      0.50       827

   micro avg       0.63      0.31      0.42      3460
   macro avg       0.55      0.25      0.32      3460
weighted avg       0.57      0.31      0.37      3460
 samples avg       0.31      0.24      0.25      3460



In [74]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    8.6s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    9.7s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    9.8s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4470457517563778


In [75]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4983, Recall: 0.4211, F1-measure: 0.4565
              precision    recall  f1-score   support

        cult       0.34      0.22      0.27       507
   flashback       0.32      0.23      0.27       549
      murder       0.60      0.58      0.59      1043
    romantic       0.45      0.35      0.39       534
    violence       0.56      0.51      0.54       827

   micro avg       0.50      0.42      0.46      3460
   macro avg       0.45      0.38      0.41      3460
weighted avg       0.48      0.42      0.45      3460
 samples avg       0.34      0.32      0.31      3460



In [76]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge'))),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2,10**-3, 10**-4,10**-5,10**-6],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print(grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  46 out of  55 | elapsed:    8.0s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    8.9s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    9.0s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.44719366389960297


In [77]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4993, Recall: 0.4113, F1-measure: 0.4510
              precision    recall  f1-score   support

        cult       0.33      0.23      0.27       507
   flashback       0.32      0.19      0.24       549
      murder       0.58      0.60      0.59      1043
    romantic       0.46      0.32      0.38       534
    violence       0.56      0.49      0.52       827

   micro avg       0.50      0.41      0.45      3460
   macro avg       0.45      0.37      0.40      3460
weighted avg       0.48      0.41      0.44      3460
 samples avg       0.34      0.31      0.30      3460



# TfidfVectorizing with char and tri grams

In [84]:
vectorize=TfidfVectorizer(ngram_range=(3,3),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 10726)   (11027, 5)
(2730, 10726)   (2730, 5)


In [85]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)



Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:    8.2s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:    9.2s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    9.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.20812916540513837


In [86]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.001),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6520, Recall: 0.1370, F1-measure: 0.2264
              precision    recall  f1-score   support

        cult       0.36      0.01      0.02       507
   flashback       0.24      0.01      0.01       549
      murder       0.68      0.30      0.41      1043
    romantic       0.55      0.06      0.11       534
    violence       0.67      0.15      0.25       827

   micro avg       0.65      0.14      0.23      3460
   macro avg       0.50      0.10      0.16      3460
weighted avg       0.54      0.14      0.20      3460
 samples avg       0.16      0.11      0.12      3460



In [87]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    8.4s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   10.2s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   10.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   10.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4293469804279494


In [89]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5421, Recall: 0.4090, F1-measure: 0.4662
              precision    recall  f1-score   support

        cult       0.44      0.07      0.12       507
   flashback       0.34      0.13      0.19       549
      murder       0.59      0.66      0.62      1043
    romantic       0.51      0.31      0.38       534
    violence       0.55      0.55      0.55       827

   micro avg       0.54      0.41      0.47      3460
   macro avg       0.49      0.34      0.37      3460
weighted avg       0.51      0.41      0.43      3460
 samples avg       0.35      0.31      0.31      3460



In [90]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:    7.5s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:    9.0s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    9.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    9.1s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4351208283706366


In [91]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4095, Recall: 0.4506, F1-measure: 0.4291
              precision    recall  f1-score   support

        cult       0.32      0.40      0.35       507
   flashback       0.29      0.31      0.30       549
      murder       0.62      0.42      0.50      1043
    romantic       0.30      0.60      0.40       534
    violence       0.53      0.51      0.52       827

   micro avg       0.41      0.45      0.43      3460
   macro avg       0.41      0.45      0.42      3460
weighted avg       0.45      0.45      0.44      3460
 samples avg       0.33      0.35      0.31      3460



# TfidfVectorizing with char and four grams

In [123]:
vectorize=TfidfVectorizer(ngram_range=(4,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)


(11027, 82528)   (11027, 5)
(2730, 82528)   (2730, 5)


In [124]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   15.4s remaining:    5.1s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   18.2s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   19.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   19.1s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.378338325107332


In [125]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))


Precision: 0.5884, Recall: 0.2855, F1-measure: 0.3845
              precision    recall  f1-score   support

        cult       0.46      0.07      0.13       507
   flashback       0.23      0.03      0.05       549
      murder       0.63      0.49      0.55      1043
    romantic       0.50      0.20      0.29       534
    violence       0.62      0.39      0.48       827

   micro avg       0.59      0.29      0.38      3460
   macro avg       0.49      0.24      0.30      3460
weighted avg       0.52      0.29      0.35      3460
 samples avg       0.28      0.22      0.23      3460



In [126]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   27.5s remaining:    6.0s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   28.0s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   30.1s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.44254741962626803


In [127]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4771, Recall: 0.3945, F1-measure: 0.4319
              precision    recall  f1-score   support

        cult       0.35      0.23      0.28       507
   flashback       0.30      0.22      0.25       549
      murder       0.59      0.52      0.56      1043
    romantic       0.38      0.38      0.38       534
    violence       0.57      0.46      0.51       827

   micro avg       0.48      0.39      0.43      3460
   macro avg       0.44      0.36      0.39      3460
weighted avg       0.47      0.39      0.43      3460
 samples avg       0.33      0.31      0.29      3460



In [128]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   24.6s remaining:    5.4s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   24.9s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   26.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.43962189082584513


In [129]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4820, Recall: 0.3913, F1-measure: 0.4320
              precision    recall  f1-score   support

        cult       0.35      0.30      0.33       507
   flashback       0.29      0.24      0.26       549
      murder       0.62      0.52      0.57      1043
    romantic       0.42      0.36      0.39       534
    violence       0.57      0.41      0.48       827

   micro avg       0.48      0.39      0.43      3460
   macro avg       0.45      0.37      0.40      3460
weighted avg       0.49      0.39      0.43      3460
 samples avg       0.33      0.31      0.29      3460



# TfidfVectorizing with char and tri,four grams

In [99]:
vectorize=TfidfVectorizer(ngram_range=(3,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93254)   (11027, 5)
(2730, 93254)   (2730, 5)


In [100]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   33.1s remaining:    7.3s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   34.6s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   36.7s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.38621552617690075


In [101]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5842, Recall: 0.2968, F1-measure: 0.3936
              precision    recall  f1-score   support

        cult       0.44      0.08      0.13       507
   flashback       0.24      0.03      0.05       549
      murder       0.64      0.51      0.56      1043
    romantic       0.51      0.22      0.30       534
    violence       0.60      0.40      0.48       827

   micro avg       0.58      0.30      0.39      3460
   macro avg       0.49      0.25      0.31      3460
weighted avg       0.52      0.30      0.36      3460
 samples avg       0.29      0.23      0.24      3460



In [130]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   18.5s remaining:    6.2s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   22.9s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   23.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   23.1s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4427437289754312


In [131]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4895, Recall: 0.4043, F1-measure: 0.4429
              precision    recall  f1-score   support

        cult       0.32      0.19      0.24       507
   flashback       0.30      0.26      0.28       549
      murder       0.59      0.56      0.58      1043
    romantic       0.46      0.30      0.36       534
    violence       0.56      0.50      0.53       827

   micro avg       0.49      0.40      0.44      3460
   macro avg       0.45      0.36      0.40      3460
weighted avg       0.48      0.40      0.44      3460
 samples avg       0.33      0.31      0.29      3460



In [134]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7,10**-8,10**-9],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done  30 out of  40 | elapsed:   16.5s remaining:    5.5s
[Parallel(n_jobs=-1)]: Done  35 out of  40 | elapsed:   19.9s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   20.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   20.1s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-08, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.436393896249056


In [135]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-08,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4597, Recall: 0.4350, F1-measure: 0.4470
              precision    recall  f1-score   support

        cult       0.34      0.23      0.28       507
   flashback       0.26      0.24      0.25       549
      murder       0.59      0.55      0.57      1043
    romantic       0.41      0.44      0.42       534
    violence       0.51      0.54      0.52       827

   micro avg       0.46      0.43      0.45      3460
   macro avg       0.42      0.40      0.41      3460
weighted avg       0.45      0.43      0.44      3460
 samples avg       0.34      0.34      0.31      3460



# TfidfVectorizing with char and bi,tri,four grams

In [106]:
vectorize=TfidfVectorizer(ngram_range=(2,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93963)   (11027, 5)
(2730, 93963)   (2730, 5)


In [107]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   34.9s remaining:    7.7s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   38.4s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   39.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.3674700024758224


In [108]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.01),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5976, Recall: 0.2717, F1-measure: 0.3735
              precision    recall  f1-score   support

        cult       0.44      0.06      0.10       507
   flashback       0.27      0.02      0.04       549
      murder       0.64      0.48      0.55      1043
    romantic       0.51      0.19      0.27       534
    violence       0.63      0.36      0.46       827

   micro avg       0.60      0.27      0.37      3460
   macro avg       0.50      0.22      0.28      3460
weighted avg       0.53      0.27      0.34      3460
 samples avg       0.27      0.21      0.22      3460



In [137]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   27.3s remaining:    6.0s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   27.9s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   30.0s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4419224936679708


In [138]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5652, Recall: 0.3821, F1-measure: 0.4559
              precision    recall  f1-score   support

        cult       0.39      0.14      0.21       507
   flashback       0.38      0.16      0.23       549
      murder       0.63      0.57      0.60      1043
    romantic       0.54      0.27      0.36       534
    violence       0.60      0.50      0.55       827

   micro avg       0.57      0.38      0.46      3460
   macro avg       0.51      0.33      0.39      3460
weighted avg       0.53      0.38      0.43      3460
 samples avg       0.34      0.29      0.29      3460



In [139]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   24.3s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   24.9s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   26.4s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4379257193438696


In [140]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4948, Recall: 0.3962, F1-measure: 0.4401
              precision    recall  f1-score   support

        cult       0.35      0.25      0.29       507
   flashback       0.30      0.21      0.25       549
      murder       0.61      0.51      0.56      1043
    romantic       0.43      0.31      0.36       534
    violence       0.56      0.51      0.54       827

   micro avg       0.49      0.40      0.44      3460
   macro avg       0.45      0.36      0.40      3460
weighted avg       0.48      0.40      0.43      3460
 samples avg       0.32      0.31      0.29      3460



# TfidfVectorizing with char and uni,bi,tri,four grams

In [113]:
vectorize=TfidfVectorizer(ngram_range=(1,4),analyzer='char')
X_train=vectorize.fit_transform(train['CleanedSynopsis'])
X_test=vectorize.transform(test['CleanedSynopsis'])

print(X_train.shape," ",y_train.shape)
print(X_test.shape," ",y_test.shape)

(11027, 93990)   (11027, 5)
(2730, 93990)   (2730, 5)


In [114]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
parameters = {
            "clf__estimator__alpha": [3,3.5,2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)
print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   36.5s remaining:    8.0s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   38.1s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   39.9s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True),
          n_jobs=-1))]
0.25840812573469113


In [115]:
clf = OneVsRestClassifier(MultinomialNB(alpha=0.001),n_jobs=-1)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.6402, Recall: 0.1821, F1-measure: 0.2835
              precision    recall  f1-score   support

        cult       0.45      0.02      0.04       507
   flashback       0.21      0.01      0.02       549
      murder       0.67      0.36      0.47      1043
    romantic       0.54      0.09      0.16       534
    violence       0.67      0.22      0.34       827

   micro avg       0.64      0.18      0.28      3460
   macro avg       0.51      0.14      0.20      3460
weighted avg       0.55      0.18      0.26      3460
 samples avg       0.20      0.14      0.16      3460



In [141]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='log',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   27.7s remaining:    6.1s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   28.2s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   30.5s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.44379329982310267


In [142]:
clf = OneVsRestClassifier(SGDClassifier(loss='log',alpha=1e-05,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.5515, Recall: 0.3777, F1-measure: 0.4484
              precision    recall  f1-score   support

        cult       0.39      0.20      0.27       507
   flashback       0.34      0.18      0.24       549
      murder       0.63      0.53      0.58      1043
    romantic       0.54      0.29      0.38       534
    violence       0.60      0.48      0.53       827

   micro avg       0.55      0.38      0.45      3460
   macro avg       0.50      0.34      0.40      3460
weighted avg       0.53      0.38      0.44      3460
 samples avg       0.32      0.29      0.28      3460



In [144]:
pipeline = Pipeline([
            ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge',n_jobs=-1))),
            ])
parameters = {
            "clf__estimator__alpha": [2,2.5,1,10**-1, 10**-2, 10**-3, 10**-4,10**-5,10**-6,10**-7],
            }

grid_search_cv = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=10,scoring='f1_micro')
grid_search_cv.fit(X_train, y_train)

print("Best parameters set:")
print (grid_search_cv.best_estimator_.steps)
print(grid_search_cv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   24.9s remaining:    5.5s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   25.1s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   27.0s finished


Best parameters set:
[('clf', OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          n_jobs=None))]
0.4426201252171625


In [145]:
clf = OneVsRestClassifier(SGDClassifier(loss='hinge',alpha=1e-06,n_jobs=-1))
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')

f1_score = 2*((precision*recall)/(precision+recall))
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1_score))
print(classification_report(y_test, predictions,target_names=names))

Precision: 0.4857, Recall: 0.3936, F1-measure: 0.4349
              precision    recall  f1-score   support

        cult       0.32      0.22      0.26       507
   flashback       0.31      0.18      0.23       549
      murder       0.59      0.56      0.57      1043
    romantic       0.41      0.29      0.34       534
    violence       0.53      0.51      0.52       827

   micro avg       0.49      0.39      0.43      3460
   macro avg       0.43      0.35      0.38      3460
weighted avg       0.47      0.39      0.42      3460
 samples avg       0.33      0.31      0.30      3460



In [148]:
from prettytable import PrettyTable
table = PrettyTable()
print("TfidfVectorization with word analyzer")
table.field_names=['Model','ngrams','f1_score','hyperparameter']
table.add_row(['MultinomialNB','1,1','0.364','0.01'])
table.add_row(['SGDClassifier(log)','1,1','0.442','1e-06'])
table.add_row(['SGDClassifier(hinge)','1,1','0.452','1e-06'])
table.add_row(['MultinomialNB','2,2','0.420','0.1'])
table.add_row(['SGDClassifier(log)','2,2','0.386','1e-06'])
table.add_row(['SGDClassifier(hinge)','2,2','0.424','1e-06'])
table.add_row(['MultinomialNB','3,3','0.272','0.01'])
table.add_row(['SGDClassifier(log)','3,3','0.344','1e-06'])
table.add_row(['SGDClassifier(hinge)','3,3','0.333','1e-06'])
table.add_row(['MultinomialNB','1,2','0.415','0.01'])
table.add_row(['SGDClassifier(log)','1,2','0.446','1e-06'])
table.add_row(['SGDClassifier(hinge)','1,2','0.454','1e-06'])
table.add_row(['MultinomialNB','1,3','0.417','0.01'])
table.add_row(['SGDClassifier(log)','1,3','0.445','1e-06'])
table.add_row(['SGDClassifier(hinge)','1,3','0.451','1e-05'])
table.add_row(['MultinomialNB','1,4','0.417','0.01'])
table.add_row(['SGDClassifier(log)','1,4','0.456','1e-06'])
table.add_row(['SGDClassifier(hinge)','1,4','0.451','1e-06'])
print(table)
print('')
table = PrettyTable()
print("TfidfVectorization with char analyzer")
table.field_names=['Model','ngrams','f1_score','hyperparameter']
table.add_row(['MultinomialNB','3,3','0.226','0.001'])
table.add_row(['SGDClassifier(log)','3,3','0.466','1e-05'])
table.add_row(['SGDClassifier(hinge)','3,3','0.429','1e-05'])
table.add_row(['MultinomialNB','4,4','0.384','0.01'])
table.add_row(['SGDClassifier(log)','4,4','0.431','1e-06'])
table.add_row(['SGDClassifier(hinge)','4,4','0.432','1e-05'])
table.add_row(['MultinomialNB','3,4','0.393','0.01'])
table.add_row(['SGDClassifier(log)','3,4','0.442','1e-05'])
table.add_row(['SGDClassifier(hinge)','3,4','0.447','1e-08'])
table.add_row(['MultinomialNB','2,4','0.373','0.01'])
table.add_row(['SGDClassifier(log)','2,4','0.455','1e-05'])
table.add_row(['SGDClassifier(hinge)','2,4','0.440','1e-05'])
table.add_row(['MultinomialNB','1,4','0.283','0.001'])
table.add_row(['SGDClassifier(log)','1,4','0.448','1e-05'])
table.add_row(['SGDClassifier(hinge)','1,4','0.434','1e-06'])
print(table)

TfidfVectorization with word analyzer
+----------------------+--------+----------+----------------+
|        Model         | ngrams | f1_score | hyperparameter |
+----------------------+--------+----------+----------------+
|    MultinomialNB     |  1,1   |  0.364   |      0.01      |
|  SGDClassifier(log)  |  1,1   |  0.442   |     1e-06      |
| SGDClassifier(hinge) |  1,1   |  0.452   |     1e-06      |
|    MultinomialNB     |  2,2   |  0.420   |      0.1       |
|  SGDClassifier(log)  |  2,2   |  0.386   |     1e-06      |
| SGDClassifier(hinge) |  2,2   |  0.424   |     1e-06      |
|    MultinomialNB     |  3,3   |  0.272   |      0.01      |
|  SGDClassifier(log)  |  3,3   |  0.344   |     1e-06      |
| SGDClassifier(hinge) |  3,3   |  0.333   |     1e-06      |
|    MultinomialNB     |  1,2   |  0.415   |      0.01      |
|  SGDClassifier(log)  |  1,2   |  0.446   |     1e-06      |
| SGDClassifier(hinge) |  1,2   |  0.454   |     1e-06      |
|    MultinomialNB     |  1,3   